In [ ]:
import gradio as gr
import requests
from datetime import datetime, timedelta

# 🔑 OpenWeatherMap API Key
API_KEY = "64be775708b8774817ae621feb910017"
API_URL = "https://api.openweathermap.org/data/2.5/forecast"

# 📦 Urdu labels mapping (basic)
def translate_weather(description):
    translation_map = {
        "clear sky": "صاف آسمان",
        "few clouds": "ہلکے بادل",
        "scattered clouds": "چھٹپٹ بادل",
        "broken clouds": "ٹوٹے ہوئے بادل",
        "shower rain": "بوندا باندی",
        "rain": "بارش",
        "thunderstorm": "آندھی اور گرج چمک",
        "snow": "برفباری",
        "mist": "دھند",
        "light rain": "ہلکی بارش",
        "moderate rain": "درمیانی بارش",
        "overcast clouds": "مکمل بادل"
    }
    return translation_map.get(description.lower(), description)

# 🔍 Get tomorrow's forecast in Urdu
def get_tomorrow_weather(city):
    try:
        params = {
            "q": city,
            "appid": API_KEY,
            "units": "metric",
            "lang": "en"  # English used for raw data
        }
        response = requests.get(API_URL, params=params)
        data = response.json()

        if data.get("cod") != "200":
            return "❌ شہر کا نام درست نہیں یا موسم کی معلومات دستیاب نہیں"

        tomorrow = (datetime.utcnow() + timedelta(days=1)).date()
        forecast = f"📅 کل ({tomorrow}) کے موسم کی پیشگوئی:\n\n"

        for entry in data["list"]:
            forecast_time = datetime.utcfromtimestamp(entry["dt"])
            if forecast_time.date() == tomorrow:
                time_str = forecast_time.strftime("%I:%M %p")
                desc_en = entry["weather"][0]["description"]
                desc_ur = translate_weather(desc_en)
                temp = entry["main"]["temp"]
                feels = entry["main"]["feels_like"]

                forecast += f"🕒 {time_str} - {desc_ur}\n🌡️ درجہ حرارت: {temp}°C، محسوس ہوتا ہے: {feels}°C\n\n"

        return forecast if "🕒" in forecast else "❌ کل کے لیے کوئی پیشگوئی دستیاب نہیں"

    except Exception as e:
        return f"❌ خرابی: {str(e)}"

# 🌐 Gradio Interface
gr.Interface(
    fn=get_tomorrow_weather,
    inputs=gr.Textbox(label="🏙️ شہر کا نام درج کریں"),
    outputs=gr.Textbox(label="🌤️ کل کے موسم کی پیشگوئی"),
    title="🌦️ موسم کی پیشگوئی (اردو)",
    description="شہر کا نام درج کریں اور کل کے موسم کی تفصیلات دیکھیں۔"
).launch(share=True)
